# Introdução à Inteligência Artificial 22/23 
## AVALIAÇÃO CONTÍNUA
<img src="Imagens\four.gif" width="120">

### Entrega até: 18 De Dezembro (23:59)

##  SPAM E NÃO-SPAM
Exercício retirado do 2º exame 21/22
<img src="Imagens\SPAMspaceship.gif" width="180">

Considere o conjunto de treino seguinte, em que os quadrados representam email de spam
e os círculos representam email não-spam.

<img src="Imagens\grelhaSirMadam.PNG" width="350">

Considere 3 intervalos em cada um dos atributos,

$[0, 12.5[, \space [12.5, 25[ \space e \space [25, ...[$ no eixo vertical

$[0, 250[, \space [250, 500[ \space e \space [500, ...[$ no eixo horizontal

Nesta situação, qual é o atributo escolhido para a raiz de uma árvore de decisão,
induzida pelo critério da menor entropia restante?

## Objetivos

Nesta avaliação vamos desenvolver código Python para resolver exercícios semelhantes ao do Spam vs não-Spam. Os objetivos são dois que implicam desenvolver duas funções de modo a:

1. Discretizar um conjunto de dados numéricos fornecendo os intervalos de discretização. Não somos obrigados a discretizar todos os atributos. Teremos que indicar quais os índices dos atributos a discretizar juntamente com os intervalos respetivos.
2. Escolher o melhor atributo para a raíz da árvore de decisão, considerando um conjunto de dados.

## Representação dos dados
#### Pontos

Vamos assumir que os dados são representados através de um dicionário em que cada ponto tem uma etiqueta que o identifica (formando as chaves do dicionário), e os valores são tuplos em que os $n-1$ primeiros elementos do tuplo serão os valores de cada um dos atributos, e o último elemento do tuplo corresponde à classe.

Começamos por representar os dados no exemplo SPAM, que guardamos na variável `pontos`. Vamos identificar os dados SPAM como *preto* e os dados não-SPAM como *branco*.

```python
pontos = {'A':(100,10,'preto'), 'B':(200,5,'branco'), 'C':(100,25,'preto'), 'D':(400,30,'preto'), \
        'E':(300,20,'preto'), 'F':(400,0,'branco'), 'G':(500,20,'branco'), 'H':(600,10,'branco'), \
        'I':(600,30,'preto'), 'J':(700,15,'preto'), 'K':(700,7,'preto'), 'L':(700,3,'branco')}
```

#### As classes e os atributos

Também temos uma lista com o nome das classes e uma com o nome dos atributos, estes pela mesma ordem que os elementos do tuplo da variável `pontos`.

```python
classes = ['branco', 'preto']
atributos = ['numero_palavras', 'sir_madam']
```
Assim, por exemplo, 'A':(100,10,'preto') é da classe *spam* e tem 'numero_palavras' = 100 e 'sir_madam' = 10.


#### E os valores dos atributos?
Como os valores dos atributos neste exemplo do SPAM ainda não estão discretizados não vamos representar os domínios de cada atributo, para já. Teremos primeiro que os discretizar!

## Discretização
A primeira função a implementar nesta avaliação contínua é `transformaPontos` que pega nos pontos do conjunto de dados e numa lista de intervalos (ordenada de acordo com a ordem nos tuplos) e devolve novos pontos.

Por exemplo, no exercício de exame SPAM, os intervalos são
```python
[0,250[, [250,500[ e [500,+[ para o atributo 'numero_palavras'.

Assim vamos ter um novo atributo discreto em que teremos 3 valores: 1, 2 e 3.
    * 1 para cada valor em [0,250[
    * 2 para cada valor em [250,500[
    * 3 para cada valor em [500,+[
```

Vamos primeiro transformar os dados de acordo com a discretização proposta.
Notem que se temos 3 intervalos, os valores do atributo convertido pertencerão a $[1,2,3]$.

Note ainda que os intervalos de um atributo são todos colocados numa lista apenas formados pelos pontos importantes. No caso do intervalo de discretização para o atributo 'numero_palavras' teremos uma lista com dois elementos, $[ 250,500 ]$, que representa os 3 intervalos ([0,250[, [250,500[ e [500,+[).

Eis o dicionário de intervalos em que as chaves são os índices dos atributos a discretizar (de acordo com a lista `atributos`definida anteriormente), e os valores são as listas de números que correspondem aos intervalos.

Por exemplo, para o exercício do SPAM:

```python
intervalos = {0:[250,500], 1:[12.5,25]}
```

Vamos converter os dados, i.e., discretizá-los, usando a função `transformaPontos`, a qual pega nos dados e na lista de intervalos e devolve um novo dicionário com os dados discretizados.

```python
data = transformaPontos(pontos,intervalos)
print(data)
```

```python
{'A': (1, 1, 'preto'), 'B': (1, 1, 'branco'), 'C': (1, 3, 'preto'), 'D': (2, 3, 'preto'), 'E': (2, 2, 'preto'), 'F': (2, 1, 'branco'), 'G': (3, 2, 'branco'), 'H': (3, 1, 'branco'), 'I': (3, 3, 'preto'), 'J': (3, 2, 'preto'), 'K': (3, 1, 'preto'), 'L': (3, 1, 'branco')}
```

#### Valores dos Atributos?
Agora já podemos representar os domínios dos atributos. Vamos usar uma lista para fazer uma correspondência entre os índices dos atributos e os índices dos seus valores. No caso do SPAM, teremos 3 valores para cada um dos atributos depois de discretizados.

```python
valores = [[1,2,3]]*2
```

## Escolha do melhor atributo

Vamos escolher o melhor atributo para a raíz da árvore de decisão através da função a implementar `escolheAtributo`, em que passamos como input:
* os dados com todos os atributos discretos, 
* a lista com os índices dos atributos para seleção, 
* a lista com os domínios de todos os atributos,
* a lista com as classes e
* um boolean a indicar se queremos o não o modo pedagógico (modo `verbose`), por omissão não queremos

e devolve como output um tuplo com:
* uma lista contendo os pares atributo/entropia, 
* o melhor par atributo/entropia (em caso de empate, respeita a ordem dos atributos!),
* uma lista de dicionários com a divisão dos dados, que satisfaz cada valor do atributo.

O output é sempre o mesmo, independentemente do modo pedagógico estar activo ou não. O que muda são as mensagens impressas ao longo do processo de seleção do atributo. A seguir apresentaremos exemplos.

**Note que a implementação do modo verbose deve respeitar o output apresentado nos exemplos, nomeadamente:**
* **terá de garantir que usa o mesmo tipo de formatação incluindo o texto apresentado,**
* **efectue todos os cálculos sem arredondamentos e arredonde apenas o resultado final com 4 casas decimais (obrigatório para que passe os testes automáticos).**

No caso do exercício de SPAM do exame, invocaríamos a função sobre a variável `data` que é o dicionário com os dados discretizados.

```python
output = escolheAtributo(data,[0,1],valores,classes)
print(output)

([(0, 0.9591), (1, 0.6887)], (1, 0.6887), [{'A': (1, 1, 'preto'), 'B': (1, 1, 'branco'), 'F': (2, 1, 'branco'), 'H': (3, 1, 'branco'), 'K': (3, 1, 'preto'), 'L': (3, 1, 'branco')}, {'E': (2, 2, 'preto'), 'G': (3, 2, 'branco'), 'J': (3, 2, 'preto')}, {'C': (1, 3, 'preto'), 'D': (2, 3, 'preto'), 'I': (3, 3, 'preto')}])
```

Com o modo pedagógico ativado teríamos o output seguinte:

```python
output = escolheAtributo(data,[0,1],valores,['branco','preto'],True)
print(output)

---> Vamos verificar o atributo com índice: 0
filtro os dados para o atributo 0 = 1
{'A': (1, 1, 'preto'), 'B': (1, 1, 'branco'), 'C': (1, 3, 'preto')}
filtro os dados para o atributo 0 = 2
{'D': (2, 3, 'preto'), 'E': (2, 2, 'preto'), 'F': (2, 1, 'branco')}
filtro os dados para o atributo 0 = 3
{'G': (3, 2, 'branco'), 'H': (3, 1, 'branco'), 'I': (3, 3, 'preto'), 'J': (3, 2, 'preto'), 'K': (3, 1, 'preto'), 'L': (3, 1, 'branco')}
Distribuição dos pontos pelas classes: [[1, 2], [1, 2], [3, 3]]
entropia([1, 2])=-1/3.log2(1/3)-2/3.log2(2/3)=0.9183
entropia([1, 2])=-1/3.log2(1/3)-2/3.log2(2/3)=0.9183
entropia([3, 3])=-3/6.log2(3/6)-3/6.log2(3/6)=1.0
entropiaMédia([[1, 2], [1, 2], [3, 3]])=3/12x0.9183+3/12x0.9183+6/12x1.0=0.9591
---> Vamos verificar o atributo com índice: 1
filtro os dados para o atributo 1 = 1
{'A': (1, 1, 'preto'), 'B': (1, 1, 'branco'), 'F': (2, 1, 'branco'), 'H': (3, 1, 'branco'), 'K': (3, 1, 'preto'), 'L': (3, 1, 'branco')}
filtro os dados para o atributo 1 = 2
{'E': (2, 2, 'preto'), 'G': (3, 2, 'branco'), 'J': (3, 2, 'preto')}
filtro os dados para o atributo 1 = 3
{'C': (1, 3, 'preto'), 'D': (2, 3, 'preto'), 'I': (3, 3, 'preto')}
Distribuição dos pontos pelas classes: [[4, 2], [1, 2], [0, 3]]
entropia([4, 2])=-4/6.log2(4/6)-2/6.log2(2/6)=0.9183
entropia([1, 2])=-1/3.log2(1/3)-2/3.log2(2/3)=0.9183
entropia([0, 3])=-0/3.log2(0/3)-3/3.log2(3/3)=0.0
entropiaMédia([[4, 2], [1, 2], [0, 3]])=6/12x0.9183+3/12x0.9183+3/12x0.0=0.6887
([(0, 0.9591), (1, 0.6887)], (1, 0.6887), [{'A': (1, 1, 'preto'), 'B': (1, 1, 'branco'), 'F': (2, 1, 'branco'), 'H': (3, 1, 'branco'), 'K': (3, 1, 'preto'), 'L': (3, 1, 'branco')}, {'E': (2, 2, 'preto'), 'G': (3, 2, 'branco'), 'J': (3, 2, 'preto')}, {'C': (1, 3, 'preto'), 'D': (2, 3, 'preto'), 'I': (3, 3, 'preto')}])
```

## Exemplo 1: Spam vs Não-spam

Após implementação das funções pedidas (`transformaPontos` e `escolheAtributos`), verifique se a sua solução para o problema Spam vs Não-spam está de acordo com o apresentado anteriormente.

## Exemplo 2: Exemplo binário
Neste exemplo não precisamos de discretizar os atributos. Retirado de um dos exames.
<img src="Imagens\binariosDados.PNG" width="200">

```python
pontos = {1:(0,0,0,0,'+'), 2:(0,0,1,0,'+'), 3:(0,1,0,1,'+'), 4:(0,1,1,0,'-'), \
        5:(1,0,0,1,'-'), 6:(1,0,1,1,'-'), 7:(1,1,0,0,'+'), 8:(1,1,1,1,'-')}
```

```python
classes = ['+','-']
atributos = ['A','B','C','D']
```

Invoquemos a função `escolheAtributo`

```python
output = escolheAtributo(pontos,[0,1,2,3],[[0,1]]*4,classes)
print(output)

([(0, 0.8113), (1, 1.0), (2, 0.8113), (3, 0.8113)], (0, 0.8113), [{1: (0, 0, 0, 0, '+'), 2: (0, 0, 1, 0, '+'), 3: (0, 1, 0, 1, '+'), 4: (0, 1, 1, 0, '-')}, {5: (1, 0, 0, 1, '-'), 6: (1, 0, 1, 1, '-'), 7: (1, 1, 0, 0, '+'), 8: (1, 1, 1, 1, '-')}])
```

Com o modo pedagógico activo:
```python
output = escolheAtributo(pontos,[0,1,2,3],[[0,1]]*4,classes,True)
print(output)

---> Vamos verificar o atributo com índice: 0
filtro os dados para o atributo 0 = 0
{1: (0, 0, 0, 0, '+'), 2: (0, 0, 1, 0, '+'), 3: (0, 1, 0, 1, '+'), 4: (0, 1, 1, 0, '-')}
filtro os dados para o atributo 0 = 1
{5: (1, 0, 0, 1, '-'), 6: (1, 0, 1, 1, '-'), 7: (1, 1, 0, 0, '+'), 8: (1, 1, 1, 1, '-')}
Distribuição dos pontos pelas classes: [[3, 1], [1, 3]]
entropia([3, 1])=-3/4.log2(3/4)-1/4.log2(1/4)=0.8113
entropia([1, 3])=-1/4.log2(1/4)-3/4.log2(3/4)=0.8113
entropiaMédia([[3, 1], [1, 3]])=4/8x0.8113+4/8x0.8113=0.8113
---> Vamos verificar o atributo com índice: 1
filtro os dados para o atributo 1 = 0
{1: (0, 0, 0, 0, '+'), 2: (0, 0, 1, 0, '+'), 5: (1, 0, 0, 1, '-'), 6: (1, 0, 1, 1, '-')}
filtro os dados para o atributo 1 = 1
{3: (0, 1, 0, 1, '+'), 4: (0, 1, 1, 0, '-'), 7: (1, 1, 0, 0, '+'), 8: (1, 1, 1, 1, '-')}
Distribuição dos pontos pelas classes: [[2, 2], [2, 2]]
entropia([2, 2])=-2/4.log2(2/4)-2/4.log2(2/4)=1.0
entropia([2, 2])=-2/4.log2(2/4)-2/4.log2(2/4)=1.0
entropiaMédia([[2, 2], [2, 2]])=4/8x1.0+4/8x1.0=1.0
---> Vamos verificar o atributo com índice: 2
filtro os dados para o atributo 2 = 0
{1: (0, 0, 0, 0, '+'), 3: (0, 1, 0, 1, '+'), 5: (1, 0, 0, 1, '-'), 7: (1, 1, 0, 0, '+')}
filtro os dados para o atributo 2 = 1
{2: (0, 0, 1, 0, '+'), 4: (0, 1, 1, 0, '-'), 6: (1, 0, 1, 1, '-'), 8: (1, 1, 1, 1, '-')}
Distribuição dos pontos pelas classes: [[3, 1], [1, 3]]
entropia([3, 1])=-3/4.log2(3/4)-1/4.log2(1/4)=0.8113
entropia([1, 3])=-1/4.log2(1/4)-3/4.log2(3/4)=0.8113
entropiaMédia([[3, 1], [1, 3]])=4/8x0.8113+4/8x0.8113=0.8113
---> Vamos verificar o atributo com índice: 3
filtro os dados para o atributo 3 = 0
{1: (0, 0, 0, 0, '+'), 2: (0, 0, 1, 0, '+'), 4: (0, 1, 1, 0, '-'), 7: (1, 1, 0, 0, '+')}
filtro os dados para o atributo 3 = 1
{3: (0, 1, 0, 1, '+'), 5: (1, 0, 0, 1, '-'), 6: (1, 0, 1, 1, '-'), 8: (1, 1, 1, 1, '-')}
Distribuição dos pontos pelas classes: [[3, 1], [1, 3]]
entropia([3, 1])=-3/4.log2(3/4)-1/4.log2(1/4)=0.8113
entropia([1, 3])=-1/4.log2(1/4)-3/4.log2(3/4)=0.8113
entropiaMédia([[3, 1], [1, 3]])=4/8x0.8113+4/8x0.8113=0.8113
([(0, 0.8113), (1, 1.0), (2, 0.8113), (3, 0.8113)], (0, 0.8113), [{1: (0, 0, 0, 0, '+'), 2: (0, 0, 1, 0, '+'), 3: (0, 1, 0, 1, '+'), 4: (0, 1, 1, 0, '-')}, {5: (1, 0, 0, 1, '-'), 6: (1, 0, 1, 1, '-'), 7: (1, 1, 0, 0, '+'), 8: (1, 1, 1, 1, '-')}])
```

## Exemplo 3: 3 classes numa grelha 2D
Adaptado de um dos exames. Queremos discretizar os dados em intervalos de 2 ($[0,2[, [2,4[, [4,6[, ...$), em ambos os atributos. E depois queremos escolher o melhor atributo para a raíz de uma árvore de decisão.
<img src="Imagens\grelhaSuja.PNG" width="200">

```python
pontos = {1:(0.5,3.5,'quadrado'), 2:(0.5,6.5,'bola'), 3:(1.5,5.5,'quadrado'), 
        4:(1.5,8.5,'bola'),5:(2.5,1.5,'quadrado'), 6:(2.5,3.5,'quadrado'),
        7:(3.5,4.5,'quadrado'), 8:(4.5,0.5,'quadrado'),9:(4.5,2.5,'quadrado'),
        10:(4.5,7.5,'bola'), 11:(6.5,1.5,'quadrado'), 12:(7.5,1.5,'x'),
        13:(7.5,3.5,'x'), 14:(7.5,5.5,'bola'), 15:(7.5,8.5,'bola'),
        16:(8.5,2.5,'x'),17:(8.5,4.5,'bola')}
```

```python
intervalos = {0:[2,4,6,8],1:[2,4,6,8]}
classes = ['quadrado','bola','x']
atributos = ['X','Y']
```

Vamos discretizar

```python
data = transformaPontos(pontos,intervalos)
print(data)

{1: (1, 2, 'quadrado'), 2: (1, 4, 'bola'), 3: (1, 3, 'quadrado'), 4: (1, 5, 'bola'), 5: (2, 1, 'quadrado'), 6: (2, 2, 'quadrado'), 7: (2, 3, 'quadrado'), 8: (3, 1, 'quadrado'), 9: (3, 2, 'quadrado'), 10: (3, 4, 'bola'), 11: (4, 1, 'quadrado'), 12: (4, 1, 'x'), 13: (4, 2, 'x'), 14: (4, 3, 'bola'), 15: (4, 5, 'bola'), 16: (5, 2, 'x'), 17: (5, 3, 'bola')}
```

Vamos seleccionar o melhor atributo:

``` python
output = escolheAtributo(data,[0,1],[[1,2,3,4,5]]*2,classes)
print(output)

([(0, 0.9626), (1, 0.7118)], (1, 0.7118), [{5: (2, 1, 'quadrado'), 8: (3, 1, 'quadrado'), 11: (4, 1, 'quadrado'), 12: (4, 1, 'x')}, {1: (1, 2, 'quadrado'), 6: (2, 2, 'quadrado'), 9: (3, 2, 'quadrado'), 13: (4, 2, 'x'), 16: (5, 2, 'x')}, {3: (1, 3, 'quadrado'), 7: (2, 3, 'quadrado'), 14: (4, 3, 'bola'), 17: (5, 3, 'bola')}, {2: (1, 4, 'bola'), 10: (3, 4, 'bola')}, {4: (1, 5, 'bola'), 15: (4, 5, 'bola')}])
```

Activando o modo pedagógico:

```python
output = escolheAtributo(data,[0,1],[[1,2,3,4,5]]*2,classes,True)
print(output)

---> Vamos verificar o atributo com índice: 0
filtro os dados para o atributo 0 = 1
{1: (1, 2, 'quadrado'), 2: (1, 4, 'bola'), 3: (1, 3, 'quadrado'), 4: (1, 5, 'bola')}
filtro os dados para o atributo 0 = 2
{5: (2, 1, 'quadrado'), 6: (2, 2, 'quadrado'), 7: (2, 3, 'quadrado')}
filtro os dados para o atributo 0 = 3
{8: (3, 1, 'quadrado'), 9: (3, 2, 'quadrado'), 10: (3, 4, 'bola')}
filtro os dados para o atributo 0 = 4
{11: (4, 1, 'quadrado'), 12: (4, 1, 'x'), 13: (4, 2, 'x'), 14: (4, 3, 'bola'), 15: (4, 5, 'bola')}
filtro os dados para o atributo 0 = 5
{16: (5, 2, 'x'), 17: (5, 3, 'bola')}
Distribuição dos pontos pelas classes: [[2, 2, 0], [3, 0, 0], [2, 1, 0], [1, 2, 2], [0, 1, 1]]
entropia([2, 2, 0])=-2/4.log2(2/4)-2/4.log2(2/4)-0/4.log2(0/4)=1.0
entropia([3, 0, 0])=-3/3.log2(3/3)-0/3.log2(0/3)-0/3.log2(0/3)=0.0
entropia([2, 1, 0])=-2/3.log2(2/3)-1/3.log2(1/3)-0/3.log2(0/3)=0.9183
entropia([1, 2, 2])=-1/5.log2(1/5)-2/5.log2(2/5)-2/5.log2(2/5)=1.5219
entropia([0, 1, 1])=-0/2.log2(0/2)-1/2.log2(1/2)-1/2.log2(1/2)=1.0
entropiaMédia([[2, 2, 0], [3, 0, 0], [2, 1, 0], [1, 2, 2], [0, 1, 1]])=4/17x1.0+3/17x0.0+3/17x0.9183+5/17x1.5219+2/17x1.0=0.9626
---> Vamos verificar o atributo com índice: 1
filtro os dados para o atributo 1 = 1
{5: (2, 1, 'quadrado'), 8: (3, 1, 'quadrado'), 11: (4, 1, 'quadrado'), 12: (4, 1, 'x')}
filtro os dados para o atributo 1 = 2
{1: (1, 2, 'quadrado'), 6: (2, 2, 'quadrado'), 9: (3, 2, 'quadrado'), 13: (4, 2, 'x'), 16: (5, 2, 'x')}
filtro os dados para o atributo 1 = 3
{3: (1, 3, 'quadrado'), 7: (2, 3, 'quadrado'), 14: (4, 3, 'bola'), 17: (5, 3, 'bola')}
filtro os dados para o atributo 1 = 4
{2: (1, 4, 'bola'), 10: (3, 4, 'bola')}
filtro os dados para o atributo 1 = 5
{4: (1, 5, 'bola'), 15: (4, 5, 'bola')}
Distribuição dos pontos pelas classes: [[3, 0, 1], [3, 0, 2], [2, 2, 0], [0, 2, 0], [0, 2, 0]]
entropia([3, 0, 1])=-3/4.log2(3/4)-0/4.log2(0/4)-1/4.log2(1/4)=0.8113
entropia([3, 0, 2])=-3/5.log2(3/5)-0/5.log2(0/5)-2/5.log2(2/5)=0.971
entropia([2, 2, 0])=-2/4.log2(2/4)-2/4.log2(2/4)-0/4.log2(0/4)=1.0
entropia([0, 2, 0])=-0/2.log2(0/2)-2/2.log2(2/2)-0/2.log2(0/2)=0.0
entropia([0, 2, 0])=-0/2.log2(0/2)-2/2.log2(2/2)-0/2.log2(0/2)=0.0
entropiaMédia([[3, 0, 1], [3, 0, 2], [2, 2, 0], [0, 2, 0], [0, 2, 0]])=4/17x0.8113+5/17x0.971+4/17x1.0+2/17x0.0+2/17x0.0=0.7118
([(0, 0.9626), (1, 0.7118)], (1, 0.7118), [{5: (2, 1, 'quadrado'), 8: (3, 1, 'quadrado'), 11: (4, 1, 'quadrado'), 12: (4, 1, 'x')}, {1: (1, 2, 'quadrado'), 6: (2, 2, 'quadrado'), 9: (3, 2, 'quadrado'), 13: (4, 2, 'x'), 16: (5, 2, 'x')}, {3: (1, 3, 'quadrado'), 7: (2, 3, 'quadrado'), 14: (4, 3, 'bola'), 17: (5, 3, 'bola')}, {2: (1, 4, 'bola'), 10: (3, 4, 'bola')}, {4: (1, 5, 'bola'), 15: (4, 5, 'bola')}])
```

## Quizz

Esta avaliação vale 1.25 em 20 e é um Quiz do Moodle com correção automática. Os testes visíveis e invisíveis relacionados com o modo pedagógico inativo (modo verbose) valem 0.4 e os testes relacionados com o modo ativo valem 0.85. Podem submeter as vezes que quiserem sem penalizações. O prazo é **18 de Dezembro às 23:59**.

<img src="Imagens\fourRabbits.gif" width="180">